In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import display, Markdown, Latex
import pickle
rng = np.random.default_rng(123)

SMALL_SIZE = 12
MEDIUM_SIZE = 14
LARGE_SIZE = 16

plt.rcdefaults()
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.size'] = MEDIUM_SIZE
plt.rcParams['axes.titlesize'] = LARGE_SIZE
plt.rcParams['axes.labelsize'] = MEDIUM_SIZE
plt.rcParams['xtick.labelsize'] = SMALL_SIZE
plt.rcParams['ytick.labelsize'] = SMALL_SIZE
plt.rcParams['legend.fontsize'] = MEDIUM_SIZE
plt.rcParams['figure.titlesize'] = LARGE_SIZE
plt.rcParams['figure.figsize'] = [7.2, 4.8]
plt.rcParams['figure.dpi'] = 60
plt.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)

from utils import Monopoly, get_random_prob


In [2]:
prob = Monopoly()

In [3]:
display(Latex(prob.general_setup()))

<IPython.core.display.Latex object>

In [4]:
display(Latex(prob.setup()))
print(prob.params)
print(prob.sol)

<IPython.core.display.Latex object>

{'Y': 100, 'alpha': 12, 'beta': 1, 'gamma': 0, 'delta': 0, 'eta': 1}
{'Q': 4.0, 'p': 8.0, 'U': 108.0, 'Profit': 24.0, 'c': 68.0, 'Q_eff': 6.0, 'p_eff': 6.0, 'U_eff': 118.0, 'Profit_eff': 18.0, 'c_eff': 64.0, 'DWL': 4.0}


In [5]:
examples = []
n_found = 0
for alpha in np.arange(6,49):
    for delta in np.arange(0, alpha-5):
        for beta in [1,2]:
            for eta in [0,1,2]:
                for Y in [100,200,300,400]:
                    params = {'Y':Y,'alpha':alpha,'beta':beta,'gamma':0,'delta':delta,'eta':eta}
                    prob = Monopoly(params)
                    if prob.check_solution():
                        examples.append(params | prob.sol)
                        
                        # Add an example with a fixed cost
                        gamma = rng.integers(2, high=prob.sol['Profit'])
                        params['gamma'] = gamma
                        prob = Monopoly(params)
                        examples.append(params | prob.sol)
                        
                        n_found+=2
                        if (n_found%1000==0):
                            print(f"{n_found} eamples found...")
examples = pd.DataFrame.from_dict(examples)
examples.to_csv("Monopoly_examples.csv", index=False, header=True)
print(f"{len(examples)} examples found.")

1000 eamples found...
1864 examples found.


In [6]:
myprob = get_random_prob(Monopoly, "Monopoly_examples.csv")
display(Latex(myprob.setup()))
print(myprob.params)
print(myprob.sol)

<IPython.core.display.Latex object>

{'Y': 200.0, 'alpha': 22.0, 'beta': 2.0, 'gamma': 0.0, 'delta': 10.0, 'eta': 2.0}
{'Q': 2.0, 'p': 18.0, 'U': 104.0, 'Profit': 12.0, 'c': 64.0, 'Q_eff': 3.0, 'p_eff': 16.0, 'U_eff': 109.0, 'Profit_eff': 9.0, 'c_eff': 52.0, 'DWL': 2.0}


In [7]:
len(examples.loc[ examples['Profit_eff']<0 ])

593

In [8]:
idx = (examples['gamma'] > examples['delta']) & (examples['Profit_eff']<0) & (examples['eta']==0)
examples.loc[idx]

,Y,alpha,beta,gamma,delta,eta,Q,p,U,Profit,c,Q_eff,p_eff,U_eff,Profit_eff,c_eff,DWL
9,100,7,1,8,1,0,3.0,4.0,104.5,1.0,88.0,6.0,1.0,118.0,-8.0,94.0,4.5
11,200,7,1,3,1,0,3.0,4.0,104.5,6.0,88.0,6.0,1.0,118.0,-3.0,94.0,4.5
13,300,7,1,3,1,0,3.0,4.0,104.5,6.0,88.0,6.0,1.0,118.0,-3.0,94.0,4.5
15,400,7,1,3,1,0,3.0,4.0,104.5,6.0,88.0,6.0,1.0,118.0,-3.0,94.0,4.5
25,100,8,1,5,2,0,3.0,5.0,104.5,4.0,85.0,6.0,2.0,118.0,-5.0,88.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,400,28,2,19,12,0,4.0,20.0,116.0,13.0,20.0,8.0,12.0,164.0,-19.0,4.0,16.0
1633,100,28,2,17,16,0,3.0,22.0,109.0,1.0,34.0,6.0,16.0,136.0,-17.0,4.0,9.0
1667,200,29,2,31,9,0,5.0,19.0,125.0,19.0,5.0,10.0,9.0,200.0,-31.0,10.0,25.0
1669,300,29,2,40,9,0,5.0,19.0,125.0,10.0,5.0,10.0,9.0,200.0,-40.0,10.0,25.0
